# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**:

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-ESCRIBE-AQUI-TU-NOMBRE") \
    .master("spark://28d4ad191d34:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 14:30:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
employees_path = "/home/jovyan/notebooks/data/employees.csv"
departments_path = "/home/jovyan/notebooks/data/departments.csv"

employees_df = spark.read.csv(employees_path, header=True, inferSchema=True)
departments_df = spark.read.csv(departments_path, header=True, inferSchema=True)

employees_df.show(5)
departments_df.show(5)


+-----------+--------------------+
|employee_id|       employee_info|
+-----------+--------------------+
|          1|{'name': 'Caitlyn...|
|          2|{'name': 'Rachel'...|
|          3|{'name': 'Carrie'...|
|          4|{'name': 'Renee',...|
|          5|{'name': 'Gabriel...|
+-----------+--------------------+
only showing top 5 rows

+-------------+--------------------+---------+
|department_id|     department_name| location|
+-------------+--------------------+---------+
|          101|     Human Resources|San Diego|
|          102|Finance and Accou...| New York|
|          103| Sales and Marketing|  Chicago|
|          104|    Data Engineering|  Zapopan|
|          105|        Data Science|  Seattle|
+-------------+--------------------+---------+
only showing top 5 rows



In [ ]:
from team_name.spark_utils import SparkUtils
from pyspark.sql.functions import col, from_json

employee_info_schema = SparkUtils.generate_schema([
    ("name", "string"),
    ("department_id", "integer"),
    ("salary", "double"),
    ("hire_date", "date")
])

parsed_df = employees_df.withColumn("parsed_info", from_json(col("employee_info"), employee_info_schema))

final_df = parsed_df.select(
    col("parsed_info.name").alias("name"),
    col("parsed_info.department_id").alias("department_id"),
    col("parsed_info.salary").alias("salary"),
    col("parsed_info.hire_date").alias("hire_date")
)

final_df.show()

+---------+-------------+---------+----------+
|     name|department_id|   salary| hire_date|
+---------+-------------+---------+----------+
|  Caitlyn|          103|115959.78|2002-06-10|
|   Rachel|          104|100820.16|2009-07-01|
|   Carrie|          105|114421.44|1998-12-10|
|    Renee|          104| 54688.13|1995-03-17|
|Gabriella|          109|106267.03|1995-02-09|
|  Kristen|          101| 88237.54|2010-11-15|
| Jonathan|          102| 39323.42|2012-06-30|
| Michelle|          101| 64262.85|2005-10-30|
| Michelle|          105|103521.88|1991-07-10|
|     Lisa|          110| 55435.93|2016-03-25|
|   Cheryl|          103| 88073.75|2020-08-21|
|  Mikayla|          107| 95192.05|2022-05-13|
|     Lisa|          104| 36032.49|2019-05-16|
|     Sean|          108| 64904.69|2021-06-06|
|   Monica|          105| 92589.97|2022-05-09|
|  Katelyn|          104|147225.58|2008-03-14|
|    Linda|          108|146632.64|2002-09-04|
|    Tammy|          109| 128860.4|2014-04-18|
|  William|  

In [ ]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

employee_info_schema = StructType([
    StructField("name", StringType(), True),
    StructField("department_id", IntegerType(), True),
    StructField("salary", DoubleType(), True),
    StructField("hire_date", DateType(), True)
])

employees_with_info_df = employees_df.withColumn(
    "employee_info_parsed", from_json(col("employee_info"), employee_info_schema)
)

employees_with_dept_id_df = employees_with_info_df.withColumn(
    "department_id", col("employee_info_parsed.department_id")
)

joined_df = employees_with_dept_id_df.join(departments_df, on="department_id", how="inner")

final_joined_df = joined_df.select(
    col("employee_info_parsed.name"),
    col("employee_info_parsed.salary"),
    col("employee_info_parsed.hire_date"),
    col("department_id"),
    col("department_name")  
)

final_joined_df.show()


+---------+---------+----------+-------------+--------------------+
|     name|   salary| hire_date|department_id|     department_name|
+---------+---------+----------+-------------+--------------------+
|  Caitlyn|115959.78|2002-06-10|          103| Sales and Marketing|
|   Rachel|100820.16|2009-07-01|          104|    Data Engineering|
|   Carrie|114421.44|1998-12-10|          105|        Data Science|
|    Renee| 54688.13|1995-03-17|          104|    Data Engineering|
|Gabriella|106267.03|1995-02-09|          109|    Customer Service|
|  Kristen| 88237.54|2010-11-15|          101|     Human Resources|
| Jonathan| 39323.42|2012-06-30|          102|Finance and Accou...|
| Michelle| 64262.85|2005-10-30|          101|     Human Resources|
| Michelle|103521.88|1991-07-10|          105|        Data Science|
|     Lisa| 55435.93|2016-03-25|          110|Corporate Strateg...|
|   Cheryl| 88073.75|2020-08-21|          103| Sales and Marketing|
|  Mikayla| 95192.05|2022-05-13|          107|  

In [ ]:
from pyspark.sql.functions import col, when

employees_with_salary_category_df = final_joined_df.withColumn(
    "salary_category",
    when(col("salary") >= 55000, "High").otherwise("Low")
)

employees_with_salary_category_df.show()

+---------+---------+----------+-------------+--------------------+---------------+
|     name|   salary| hire_date|department_id|     department_name|salary_category|
+---------+---------+----------+-------------+--------------------+---------------+
|  Caitlyn|115959.78|2002-06-10|          103| Sales and Marketing|           High|
|   Rachel|100820.16|2009-07-01|          104|    Data Engineering|           High|
|   Carrie|114421.44|1998-12-10|          105|        Data Science|           High|
|    Renee| 54688.13|1995-03-17|          104|    Data Engineering|            Low|
|Gabriella|106267.03|1995-02-09|          109|    Customer Service|           High|
|  Kristen| 88237.54|2010-11-15|          101|     Human Resources|           High|
| Jonathan| 39323.42|2012-06-30|          102|Finance and Accou...|            Low|
| Michelle| 64262.85|2005-10-30|          101|     Human Resources|           High|
| Michelle|103521.88|1991-07-10|          105|        Data Science|         

In [ ]:
from pyspark.sql import functions as F

high_salary_df = employees_with_salary_category_df.filter(col("salary_category") == "High")
low_salary_df = employees_with_salary_category_df.filter(col("salary_category") == "Low")

high_salary_avg_df = high_salary_df.groupBy("department_name").agg(
    F.avg("salary").alias("avg_salary")
)

low_salary_avg_df = low_salary_df.groupBy("department_name").agg(
    F.avg("salary").alias("avg_salary")
)

high_salary_avg_df.show()
low_salary_avg_df.show()

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|102741.38324414717|
| Sales and Marketing|100839.65275449108|
|    Data Engineering|101626.29492163012|
|Research and Deve...|  98714.3003086419|
|Finance and Accou...|100731.07877887784|
|    Customer Service|101585.01600000002|
|               Legal|  99366.3129102167|
|        Data Science|101903.63710344829|
|          Operations|100169.65621722837|
|     Human Resources|104999.43191489363|
+--------------------+------------------+

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|41590.741833333326|
| Sales and Marketing| 41150.40277777778|
|    Data Engineering| 41358.50794117647|
|Finance and Accou...|42740.952888888874|
|Research and Deve...| 41426.43521126761|
|    Customer Service|42644.472021276604|
|               Legal| 41160.2661

In [ ]:
top_5_high_salary_df = high_salary_df.orderBy(col("salary").desc()).limit(5)
top_5_low_salary_df = low_salary_df.orderBy(col("salary").desc()).limit(5)

top_5_high_salary_df.show()
top_5_low_salary_df.show()

In [9]:
from pyspark.sql.functions import current_date, datediff, col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

employee_info_schema = StructType([
    StructField("name", StringType(), True),
    StructField("department_id", IntegerType(), True),
    StructField("salary", IntegerType(), True),
    StructField("hire_date", DateType(), True)
])

employees_with_hire_date_df = employees_df.withColumn(
    "employee_info", from_json(col("employee_info"), employee_info_schema)
)
employees_with_years_df = employees_with_hire_date_df.withColumn(
    "years_in_company", (datediff(current_date(), col("employee_info.hire_date")) / 365).cast("int")
)

employees_with_years_df.show()

max_years_in_company = employees_with_years_df.agg({"years_in_company": "max"}).collect()[0][0]
employees_with_max_years = employees_with_years_df.filter(col("years_in_company") == max_years_in_company)
employee_count = employees_with_max_years.count()

employees_with_max_years.show()

print(f"Number of employees with max years: {employee_count}")

+-----------+--------------------+----------------+
|employee_id|       employee_info|years_in_company|
+-----------+--------------------+----------------+
|          1|{Caitlyn, 103, NU...|              22|
|          2|{Rachel, 104, NUL...|              15|
|          3|{Carrie, 105, NUL...|              26|
|          4|{Renee, 104, NULL...|              30|
|          5|{Gabriella, 109, ...|              30|
|          6|{Kristen, 101, NU...|              14|
|          7|{Jonathan, 102, N...|              12|
|          8|{Michelle, 101, N...|              19|
|          9|{Michelle, 105, N...|              33|
|         10|{Lisa, 110, NULL,...|               8|
|         11|{Cheryl, 103, NUL...|               4|
|         12|{Mikayla, 107, NU...|               2|
|         13|{Lisa, 104, NULL,...|               5|
|         14|{Sean, 108, NULL,...|               3|
|         15|{Monica, 105, NUL...|               2|
|         16|{Katelyn, 104, NU...|              17|
|         17

In [11]:
sc.stop()